In [3]:
from django_for_jupyter import init_django
init_django()

In [4]:
# from selenium import webdriver
# import chromedriver_autoinstaller


# chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
#                                       # and if it doesn't exist, download it automatically,
#                                       # then add chromedriver to path

# driver = webdriver.Chrome()
# driver.get("http://www.python.org")
# assert "Python" in driver.title


INFO:root:Chromedriver is already installed.


In [ ]:
import pandas as pd
import numpy as np

from django.db.models import F, Sum, Count
from precios.models import SiteURLResults, Articulos, Unifica, Marcas, Vendedores, AllPalabras, ReemplazaPalabras
from precios.pi_get import obtener_grados, get_palabras_con_numychar

from precios.pi_rules import (
    intenta_marca,
    imprime_reglas,
    generate_rules,
    create_PD_From,
)
from fuzzywuzzy import fuzz, process

In [ ]:
# # !pip install extruct

# import json

# import requests
# from bs4 import BeautifulSoup


# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# import chromedriver_autoinstaller
# try:
#     chromedriver_autoinstaller.install() 
# except:
#     print('Sin conexcion')

# import extruct as ex

# url = 'https://www.santaisabel.cl/arvejas-252-g/'

# def get_driver():
#     options = Options()
#     options.add_argument('--headless')
#     driver = webdriver.Chrome(options=options)
#     return driver

# def get_source(driver, url):
#     driver.get(url)
#     return driver.page_source

# def get_json(source):
#     return ex.extract(source, syntaxes=['json-ld'])


# def get_ld_json(url: str) -> dict:
#     parser = "html.parser"
#     req = requests.get(url)
#     soup = BeautifulSoup(req.text, parser)
#     resultados = soup.select("script", {"type":"application/ld+json"})
#     for resultado in resultados:
#         print("".join(resultado.text))

#     return json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

# driver = get_driver()
# source = get_source(driver, url)
# datos = get_json(source)
# print(datos)

# response = requests.get(url).text
# jsonData = ex.extract(response, syntaxes=['json-ld'])['json-ld']
# df = pd.json_normalize(jsonData)

# print(df.to_string())

# # datos = get_ld_json(url=url)


In [ ]:
# Articulos.objects.all().delete()
# Unifica.objects.all().update(contador=0)
# AllPalabras.objects.all().update(contador=0)
# ReemplazaPalabras.objects.all().update(contador=0)
# Unifica.objects.filter(automatico=True, contador=0).delete()  ### Borra reglas sin uso

In [ ]:
# marcas_busco = ('santa-helena','tres-palos','vionelli','milo','aperol','livenza', 'campari', 'rossard','tres-torres','alto-del-carmen','nescafé','alusweet','alusa','powerade', 'becker', 'aperol')
marcas_busco = ('zacapa',)
# marcas_busco = None
marcas = Marcas.objects.filter(es_marca=True)
if marcas_busco:
    marcas = marcas.filter(slug__in=marcas_busco)
else:
    marcas = marcas[0:100]

for marca in marcas:
    print(marca.slug, marca.id)
    intenta_marca(marca.id, True)

In [5]:
import requests
from bs4 import BeautifulSoup

def find_different_css_elements(url1, url2):
    response1 = requests.get(url1)
    response2 = requests.get(url2)

    print(response1)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        # print(soup1)
        
        css_elements1 = set()
        css_elements2 = set()
        
        for element in soup1.find_all(True):
            print(element.name)
            css_elements1.add(element.name)
        
        for element in soup2.find_all(True):
            css_elements2.add(element.name)
        
        different_css_elements = css_elements1.symmetric_difference(css_elements2)
        
        return different_css_elements
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.devop.cl/precios/article/jumbo-artesanal-pantrucas-granel-10'
url_product2 = 'https://www.devop.cl/precios/article/jumbo-artesanal-panzottis-espinaca-6500-10'

different_css = find_different_css_elements(url_product1, url_product2)
if different_css:
    print('Elementos CSS diferentes entre las dos URLs:')
    for element in different_css:
        print(element)

<Response [200]>
html
head
meta
meta
title
meta
meta
meta
meta
meta
meta
meta
link
meta
link
link
link
link
link
link
link
link
link
link
link
script
script
script
script
script
noscript
body
section
div
div
button
div
div
h5
button
div
div
div
label
input
label
input
div
div
label
select
div
div
label
select
div
div
label
select
div
strong
div
a
i
a
i
a
i
a
i
header
div
h2
img
a
nav
ul
li
a
span
li
a
span
i
ul
a
span
a
span
li
a
span
i
ul
a
span
a
span
li
a
span
i
ul
a
span
a
span
a
span
a
span
a
span
i
main
script
script
script
section
div
div
h2
ol
li
a
li
a
li
a
section
div
div
div
div
div
div
a
div
img
div
div
h5
a
div
h5
span
div
div
span
br
span
div
div
div
div
button
i
div
div
div
ol
li
div
div
div
div
span
a
i
span
p
p
a
div
h5
span
div
div
div
div
div
div
div
a
i
div
footer
div
main
section
div
div
div
h3
div
form
div
div
div
label
span
select
option
option
option
div
div
label
textarea
input
input
input
div
button
div
p
div
div
i
a
i
a
i
a
i
a
i
a
i
a
div
div
strong
span
p
a

In [8]:
import requests
from bs4 import BeautifulSoup

def find_different_content_tags(url1, url2):
    response1 = requests.get(url1)
    response2 = requests.get(url2)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        
        different_content_tags = set()
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            if tag1.text.strip() != tag2.text.strip():
                print(tag1.name, ' -> ',tag1.text.strip())
                different_content_tags.add(tag1.name)
        
        return different_content_tags
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.devop.cl/precios/article/jumbo-artesanal-pantrucas-granel-10'
url_product2 = 'https://www.devop.cl/precios/article/jumbo-artesanal-panzottis-espinaca-6500-10'

different_content_tags = find_different_content_tags(url_product1, url_product2)
if different_content_tags:
    print('Etiquetas con contenido diferente entre las dos URLs:')
    for tag in different_content_tags:
        print(tag)

html  ->  Jumbo Artesanal: Pantrucas Granel 
             
             
            


      Es necesario Javascript funcionando
  Indique Cobertura
CoberturaDatos de todo el sistemaDatos de un paísPaísRegiónComunaHomePrecios BuscarMarcasMi CuentaIniciar sesiónRegistrarseAcercaPlanes para personas que buscanPlanes para empresas que ofrecenCoberturas de despachoEstado indexacionAntiguedad registrosJumbo Artesanal: Pantrucas Granel
HomeMarcasJumbo Artesanal
Jumbo Artesanal

Pantrucas Granel
$ 2.690 1 Supermercados
Precios actuales
Jumbo in stock
Hace 2 días, 3 horas atrás.
$ 2.690 
Gráfica
Que le parce este sitio ?
El sitio le sirve ?*sinoun poco
Que me puede indicar
EnviarGraciasInforme de ErroresVersionesContactoProteccion de datosPreguntas frecuentesCondiciones
© Copyright 2022 - 2023 Devop. Todos los derechos reservados
Versión en desarrollo #0.24
Designed by BootstrapMade
head  ->  Jumbo Artesanal: Pantrucas Granel 
             
             
            


      Es necesario Java

In [17]:
import requests
from bs4 import BeautifulSoup

def find_different_content_with_classes(url1, url2):
    response1 = requests.get(url1)
    response2 = requests.get(url2)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        
        different_content_tags = []
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            if tag1.text.strip() != tag2.text.strip():
                
                tag_classes = tag1.get('class')
                different_content_tags.append((tag1.name, tag_classes, tag1.text.strip(), tag2.text.strip()))
        
        return different_content_tags
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un '

different_content_and_classes = find_different_content_with_classes(url_product1, url_product2)
if different_content_and_classes:
    print('Etiquetas con contenido diferente y sus clases CSS:')
    for tag_name, tag_classes, contenido1, contenido2 in different_content_and_classes:
        print(f'Tag: {tag_name}')
        if tag_classes:
            print(f'Clases: {", ".join(tag_classes)}')
        else:
            print('Sin clases')
        if contenido1:
            print('contiene1: ',contenido1)
            print('contiene2: ',contenido2)
        print('-' * 30)

Etiquetas con contenido diferente y sus clases CSS:
Tag: html
Sin clases
contiene1:  Pan libanes Spagnolia chico 6 un | Unimarc

CategoríasOfertas¡Hola!Inicia sesiónMisCompras¿Dónde quieres recibir tu compra?InicioUnimarcUnimarcspagnoliaPan libanes Spagnolia chico 6 un250 gSku: 000000000000619558-UN$1.490($5.960 x kg)AgregarInformación del productoDescripciónPan libanes Spagnolia chico, 6 unLoading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...
contiene2:  Pan libanes Spagnolia integral chico 6 un | Unimarc

CategoríasOfertas¡Hola!Inicia sesiónMisCompras¿Dónde quieres recibir tu compra?InicioUnimarcUnimarcspagnoliaPan libanes Spagnolia integral chico 6 un250 gSku: 000000000000635333-UN$1.490($5.960 x kg)AgregarInformación del productoDescripciónPan libanes Spagnolia integral chico, 6 unLoading...Loading

In [19]:
import json

def find_product_ld_json(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ld_json_tags = soup.find_all('script', type='application/ld+json')
        
        product_data = []
        
        for tag in ld_json_tags:
            try:
                ld_json = json.loads(tag.string)
                if isinstance(ld_json, dict) and ld_json.get('@type') == 'Product':
                    product_data.append(ld_json)
            except json.JSONDecodeError:
                pass
        
        return product_data
    else:
        print('No se pudo acceder a la página:', url)
        return None
# URL de ejemplo de un producto en un sitio web
product_url = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'

product_ld_json = find_product_ld_json(product_url)
if product_ld_json:
    print('Información de producto con tipo "@type":"Product":')
    for product in product_ld_json:
        print(json.dumps(product, indent=4))
        print('-' * 30)

Información de producto con tipo "@type":"Product":
{
    "@context": "https://schema.org/",
    "@type": "Product",
    "name": "pan libanes spagnolia chico 6 un",
    "image": [
        "https://unimarc.vtexassets.com/arquivos/ids/187332/Pan-Libanes-Chico-Spagnolia-6-Un-1-17767.jpg?v=637015711380830000",
        "https://unimarc.vtexassets.com/arquivos/ids/187428/Pan-Libanes-Chico-Spagnolia-6-Un-2-17767.jpg?v=637015713344270000"
    ],
    "description": "Pan libanes Spagnolia chico, 6 un",
    "sku": "14748",
    "brand": {
        "@type": "Brand",
        "name": "Spagnolia"
    },
    "offers": {
        "@type": "Offer",
        "url": "https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un",
        "priceCurrency": "CLP",
        "price": 1490,
        "availability": "https://schema.org/inStock"
    }
}
------------------------------


In [24]:
import requests
from bs4 import BeautifulSoup
import json

def find_product_details_in_ld_json(url):

    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ld_json_tags = soup.find_all('script', type='application/ld+json')
        
        for tag in ld_json_tags:
            try:
                ld_json = json.loads(tag.string)
                if isinstance(ld_json, dict) and ld_json.get('@type') == 'Product':
                    product_details = {
                        'nombre': ld_json.get('name'),
                        'image': ld_json.get('image'),
                        'sku': ld_json.get('sku'),
                        'description': ld_json.get('description'),
                        'marca': ld_json.get('brand', {}).get('name'),
                        'moneda_precio': ld_json.get('offers', {}).get('priceCurrency'),
                        'precio': ld_json.get('offers', {}).get('price')
                    }
                    return product_details
            except json.JSONDecodeError:
                pass
        
        return None
    else:
        print('No se pudo acceder a la página:', url)
        return None

# URL de ejemplo de un producto en un sitio web
product_url = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'

detalles_producto = find_product_details_in_ld_json(product_url)
# find_product_name_in_ld_json(product_url)
if detalles_producto:
    print('Detalles del producto:')
    print('Nombre:', detalles_producto['nombre'])
    print('image:', detalles_producto['image'])
    print('description:', detalles_producto['description'])
    print('SKU:', detalles_producto['sku'])
    print('Marca:', detalles_producto['marca'])
    print('Moneda Precio:', detalles_producto['moneda_precio'])
    print('Precio:', detalles_producto['precio'])
else:
    print('No se pudieron encontrar los detalles del producto en la página.')



Detalles del producto:
Nombre: pan libanes spagnolia chico 6 un
image: ['https://unimarc.vtexassets.com/arquivos/ids/187332/Pan-Libanes-Chico-Spagnolia-6-Un-1-17767.jpg?v=637015711380830000', 'https://unimarc.vtexassets.com/arquivos/ids/187428/Pan-Libanes-Chico-Spagnolia-6-Un-2-17767.jpg?v=637015713344270000']
description: Pan libanes Spagnolia chico, 6 un
SKU: 14748
Marca: Spagnolia
Moneda Precio: CLP
Precio: 1490


In [25]:
import requests
from bs4 import BeautifulSoup
import json

def find_different_content_with_classes(url1, url2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response1 = requests.get(url1, headers=headers)
    response2 = requests.get(url2, headers=headers)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        
        different_content_tags = []
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            try:
                if tag1.text.strip() != tag2.text.strip():
                    tag_classes = tag1.get('class')
                    different_content_tags.append((tag1.name, tag_classes, tag1.text.strip(), tag2.text.strip()))
            except AttributeError:
                pass
        
        return different_content_tags
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un'

different_content_and_classes = find_different_content_with_classes(url_product1, url_product2)
if different_content_and_classes:
    print('Etiquetas con contenido diferente y sus clases CSS:')
    for tag_name, tag_classes, contenido1, contenido2 in different_content_and_classes:
        print(f'Tag: {tag_name}')
        if tag_classes:
            print(f'Clases: {", ".join(tag_classes)}')
        else:
            print('Sin clases')
        if contenido1:
            print('Contenido 1:', contenido1)
            print('Contenido 2:', contenido2)
        print('-' * 30)

Etiquetas con contenido diferente y sus clases CSS:
Tag: html
Sin clases
Contenido 1: Pan libanes Spagnolia chico 6 un | Unimarc

CategoríasOfertas¡Hola!Inicia sesiónMisCompras¿Dónde quieres recibir tu compra?InicioUnimarcUnimarcspagnoliaPan libanes Spagnolia chico 6 un250 gSku: 000000000000619558-UN$1.490($5.960 x kg)AgregarInformación del productoDescripciónPan libanes Spagnolia chico, 6 unLoading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...Loading...
Contenido 2: Pan libanes Spagnolia integral chico 6 un | Unimarc

CategoríasOfertas¡Hola!Inicia sesiónMisCompras¿Dónde quieres recibir tu compra?InicioUnimarcUnimarcspagnoliaPan libanes Spagnolia integral chico 6 un250 gSku: 000000000000635333-UN$1.490($5.960 x kg)AgregarInformación del productoDescripciónPan libanes Spagnolia integral chico, 6 unLoading...Loadi

In [26]:
import requests
from bs4 import BeautifulSoup

def find_tag_values(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tag_values = {}
        
        for tag in soup.find_all():
            tag_name = tag.name
            tag_classes = tag.get('class')
            tag_content = tag.text.strip()
            
            if tag_name and tag_classes:
                tag_values[tag_name] = {'classes': tag_classes, 'content': tag_content}
        
        return tag_values
    else:
        print('No se pudo acceder a la página:', url)
        return None

# URL de ejemplo de un producto en un sitio web
url_product = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'

tag_values = find_tag_values(url_product)
if tag_values:
    print('Valores de las etiquetas y clases en la URL:')
    for tag_name, data in tag_values.items():
        print(f'Tag: {tag_name}')
        print(f'Clases: {", ".join(data["classes"])}')
        print(f'Contenido: {data["content"]}')
        print('-' * 30)

Valores de las etiquetas y clases en la URL:
Tag: div
Clases: baseContainer_container__TSgMX, baseContainer_justify-start___sjrG, baseContainer_align-start__6PKCY, baseContainer_flex-direction--row__4HZkU, baseContainer_absolute-default--topLeft__lN1In
Contenido: Loading...
------------------------------
Tag: header
Clases: HeaderPrototype_main-header__hVEqN, HeaderPrototype_show__tkQlQ
Contenido: CategoríasOfertas¡Hola!Inicia sesiónMisCompras
------------------------------
Tag: a
Clases: Link_link___5dmQ, Link_link--none__BjwPj, Link_link--regular__kQISW, Link_link--base__C5fOX
Contenido: spagnolia
------------------------------
Tag: span
Clases: spacer
Contenido: 
------------------------------
Tag: p
Clases: Text_text__cB7NM, Text_text--left__1v2Xw, Text_text--flex__F7yuI, Text_text--regular__KSs6J, Text_text--lg__GZWsa, Text_text--gray__r4RdT, Text_text__cursor--auto__cMaN1, Text_text--none__zez2n
Contenido: Pan libanes Spagnolia chico, 6 un
------------------------------
Tag: inpu

In [28]:
import requests
from bs4 import BeautifulSoup

def find_different_content_with_classes(url1, url2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response1 = requests.get(url1, headers=headers)
    response2 = requests.get(url2, headers=headers)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        
        different_content_tags = []
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            try:
                if tag1.text.strip() != tag2.text.strip():
                    tag_classes = tag1.get('class')
                    different_content_tags.append((tag1.name, tag_classes))
            except AttributeError:
                pass
        
        return different_content_tags
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

def find_tag_values(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tag_values = {}
        
        for tag in soup.find_all():
            tag_name = tag.name
            tag_classes = tag.get('class')
            tag_content = tag.text.strip()
            
            if tag_name and tag_classes:
                tag_values[tag_name] = {'classes': tag_classes, 'content': tag_content}
        
        return tag_values
    else:
        print('No se pudo acceder a la página:', url)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un'

different_content_and_classes = find_different_content_with_classes(url_product1, url_product2)
if different_content_and_classes:
    print('Etiquetas con contenido diferente y sus clases CSS en la primera URL:')
    for tag_name, tag_classes in different_content_and_classes:
        print(f'Tag: {tag_name}')
        if tag_classes:
            print(f'Clases: {", ".join(tag_classes)}')
            tag_values = find_tag_values(url_product1)
            if tag_name in tag_values:
                print(f'Valores - Clases: {tag_values[tag_name]["classes"]}')
                print(f' Contenido: {tag_values[tag_name]["content"]}')
        else:
            print('Sin clases')
        print('-' * 30)


Etiquetas con contenido diferente y sus clases CSS en la primera URL:
Tag: html
Sin clases
------------------------------
Tag: head
Sin clases
------------------------------
Tag: head
Sin clases
------------------------------
Tag: title
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: body
Sin clases
------------------------------
Tag: div
Sin clases
------------------------------
Tag: div
Clases: baseContainer_container__TSgMX, baseContainer_is-wrap___tHwM, baseContainer_justify-start___sjrG, baseContainer_align-start__6PKCY, baseContainer_absolute-default--topLeft__lN1In
Valores - Clases: ['baseContainer_container__TSgMX', 'baseContainer_justify-start___sjrG', 'baseContainer_align-start__6PKCY', 'baseContainer_flex-direction--row__4HZkU', 'baseContainer_absolute-default--topLeft__lN1In']
 Contenido: Loading...
----------------

In [30]:
import requests
from bs4 import BeautifulSoup

def find_different_img_attributes(url1, url2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response1 = requests.get(url1, headers=headers)
    response2 = requests.get(url2, headers=headers)
    
    if response1.status_code == 200 and response2.status_code == 200:
        soup1 = BeautifulSoup(response1.content, 'html.parser')
        soup2 = BeautifulSoup(response2.content, 'html.parser')
        
        different_img_attributes = []
        
        for img1, img2 in zip(soup1.find_all('img'), soup2.find_all('img')):
            print(img1)
            if img1.attrs != img2.attrs:
                different_img_attributes.append((img1.attrs, img2.attrs))
        
        return different_img_attributes
    else:
        print('No se pudo acceder a una de las páginas:', url1, url2)
        return None

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un'

different_img_attrs = find_different_img_attributes(url_product1, url_product2)
if different_img_attrs:
    print('Atributos de las etiquetas <img> diferentes entre las dos URLs:')
    for img_attrs1, img_attrs2 in different_img_attrs:
        print('Atributos en la primera URL:', img_attrs1)
        print('Atributos en la segunda URL:', img_attrs2)
        print('-' * 30)


In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def find_different_img_attributes(url1, url2):
    driver = webdriver.Chrome()  # Asegúrate de tener ChromeDriver instalado y en el PATH
    
    try:
        driver.get(url1)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img')))
        
        soup1 = BeautifulSoup(driver.page_source, 'html.parser')
        
        driver.get(url2)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img')))
        
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        
        different_img_attributes = []
        
        for img1, img2 in zip(soup1.find_all('img'), soup2.find_all('img')):
            if img1.attrs != img2.attrs:
                different_img_attributes.append((img1.attrs, img2.attrs))
        
        return different_img_attributes
    finally:
        driver.quit()

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un'

different_img_attrs = find_different_img_attributes(url_product1, url_product2)
if different_img_attrs:
    print('Atributos de las etiquetas <img> diferentes entre las dos URLs:')
    for img_attrs1, img_attrs2 in different_img_attrs:
        print('Atributos en la primera URL:', img_attrs1)
        print('Atributos en la segunda URL:', img_attrs2)
        print('-' * 30)


Atributos de las etiquetas <img> diferentes entre las dos URLs:
Atributos en la primera URL: {'alt': 'Pan libanes Spagnolia chico 6 un', 'class': [], 'height': '100%', 'loading': 'eager', 'title': 'Pan libanes Spagnolia chico 6 un', 'width': '100%', 'src': 'https://unimarc.vtexassets.com/arquivos/ids/187332/Pan-Libanes-Chico-Spagnolia-6-Un-1-17767.jpg?v=637015711380830000', 'style': 'height: 100%; max-width: 100%;'}
Atributos en la segunda URL: {'alt': 'Pan libanes Spagnolia integral chico 6 un', 'class': [], 'height': '100%', 'loading': 'eager', 'title': 'Pan libanes Spagnolia integral chico 6 un', 'width': '100%', 'src': 'https://unimarc.vtexassets.com/arquivos/ids/187346/PAN-LIBANES-INTEGRAL-CHICO-SPAGNOLIA-6UN-1-61735.jpg?v=637015711415430000', 'style': 'height: 100%; max-width: 100%;'}
------------------------------
Atributos en la primera URL: {'alt': 'Pan libanes Spagnolia chico 6 un', 'class': [], 'height': '100%', 'loading': 'eager', 'title': 'Pan libanes Spagnolia chico 6 un'

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def find_different_content_with_classes(url1, url2):
    driver = webdriver.Chrome()  # Asegúrate de tener ChromeDriver instalado y en el PATH
    
    try:
        driver.get(url1)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))
        soup1 = BeautifulSoup(driver.page_source, 'html.parser')
        
        driver.get(url2)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        
        different_content_tags = []
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            try:
                if tag1.text.strip() != tag2.text.strip():
                    tag_classes = tag1.get('class')
                    different_content_tags.append((tag1.name, tag_classes))
            except AttributeError:
                pass
        
        return different_content_tags
    finally:
        driver.quit()

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un'

different_content_and_classes = find_different_content_with_classes(url_product1, url_product2)
if different_content_and_classes:
    print('Etiquetas con contenido diferente y sus clases CSS en la primera URL:')
    for tag_name, tag_classes in different_content_and_classes:
        print(f'Tag: {tag_name}')
        if tag_classes:
            print(f'Clases: {", ".join(tag_classes)}')
        else:
            print('Sin clases')
        print('-' * 30)


Etiquetas con contenido diferente y sus clases CSS en la primera URL:
Tag: html
Sin clases
------------------------------
Tag: head
Sin clases
------------------------------
Tag: title
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: meta
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: link
Sin clases
------------------------------
Tag: script
Sin clases
------------------------------
Tag: link
Sin clases
------------------------------
Tag: body
Sin clases
------------------------------
Tag: div
Sin clases
------------------------------
Tag: div
Clases: baseContainer_container__TSgMX, baseContainer_is-wrap___tHwM, baseContainer_justify-start___sjrG, baseContainer_align-start__6PKCY, baseContainer_absolute-default--topLeft__lN1In
------------------------------
Tag: main
Clases: baseContainer_container__TSgMX, baseContainer_justify

In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def find_different_content_with_classes(url1, url2):
    driver = webdriver.Chrome()  # Asegúrate de tener ChromeDriver instalado y en el PATH
    
    try:
        driver.get(url1)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))
        soup1 = BeautifulSoup(driver.page_source, 'html.parser')
        
        driver.get(url2)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        
        different_content_tags = []
        
        for tag1, tag2 in zip(soup1.find_all(), soup2.find_all()):
            try:
                if tag1.text.strip() != tag2.text.strip():
                    tag_classes = tag1.get('class')
                    different_content_tags.append((tag1.name, tag_classes, tag1.text.strip(), tag2.text.strip()))
            except AttributeError:
                pass
        
        return different_content_tags
    finally:
        driver.quit()

# URLs de ejemplo de dos productos diferentes en un sitio web
url_product1 = 'https://www.unimarc.cl/product/pan-libanes-chico-spagnolia-6-un'
url_product2 = 'https://www.unimarc.cl/product/pan-libanes-integral-chico-spagnolia-6un '

different_content_and_classes = find_different_content_with_classes(url_product1, url_product2)
if different_content_and_classes:
    print('Etiquetas con contenido diferente y sus clases CSS:')
    for tag_name, tag_classes, contenido1, contenido2 in different_content_and_classes:
        print(f'Tag: {tag_name}')
        if tag_classes:
            print(f'Clases: {", ".join(tag_classes)}')
        else:
            print('Sin clases')
        if contenido1:
            print('contiene1: ', contenido1)
            print('contiene2: ', contenido2)
        print('-' * 30)


Etiquetas con contenido diferente y sus clases CSS:
Tag: html
Sin clases
contiene1:  Pan libanes Spagnolia chico 6 un | Unimarc

¡Hola!Inicia sesiónOfertasMis comprasCuponesCatálogoLocales y horariosUnidos¿Dónde quieres recibir tu compra?👋¡Hola! Todos nuestros despachos son gratis por compras sobre $40.000. ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Comparte, regala y ayuda con Unidos: Regala una Giftcard por un 18 más solidario ¡Aporta ahora!👋¡Hola! Todos nuestros despachos son gratis por compras sobre $40.000. ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Comparte, regala y ayuda con Unidos: Regala una Giftcard por un 18 más solidario ¡Aporta ahora!👋¡Hola! Todos nuestros despachos son gratis por compras sobre $40.000. ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Comparte, regala y ayuda con Unidos: Regala una Giftcard por un 18 más solidario ¡Aporta ahora!ProductoDescripciónspagnoliaPan libanes Spagnolia chico 6 un250 gSku: 000000000000619558-UN$1.490($5.960 x kg)Loading...Loading...Lo

In [35]:
import urllib.parse

encoded_data = "\\x16\\x03\\x01\\x00\\x85\\x01\\x00\\x00\\x81\\x03\\x03\\x7F\\x09\\xBB\\x00\\xAD\\xBB\\x983\\xBA\\x04a\\x032\\xA8\\xFAP\\x8A\\x89MA \\xE2\\xC6TT_\\x85|<\\x02\\x18\\x9F\\x00\\x00 \\xC0/\\xC00\\xC0+\\xC0,\\xCC\\xA8\\xCC\\xA9\\xC0\\x13\\xC0\\x09\\xC0\\x14\\xC0"
decoded_data = bytes.fromhex(encoded_data.replace("\\x", "")).decode("utf-8")

print(decoded_data)

ValueError: non-hexadecimal number found in fromhex() arg at position 49